In [ ]:
import sys
sys.path.append('/data/ECS/PythonTutorial/')
import log
log.topic_id = '24_VeriGorsellestirme-matplotlib'

## Veri Görselleştirme

### matplotlib

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
print(matplotlib.__version__)

In [ ]:
import os
import numpy as np
import pandas as pd
import cx_Oracle

def connect_db(user_name=None, password=None):
    try:
        with open('/data/home/alperayd/oracle.txt','r') as f:
            r = f.read()
            user_name = r.split(';')[0]
            password = r.split(';')[1]
        
        os.environ['ORACLE_HOME'] = '/data/app/oracle/product/12.2.0/client_1'
        host = "pgarsnb1rw.fw.garanti.com.tr:4520/pgarsnb1b"
        conn = cx_Oracle.connect(user_name, password, host, encoding='UTF-8')
        return conn
    except Exception as ex:
        print(f"bağlantı sağlanamadı, \nHata: {ex}")
        return None
    
    
sql = """
    select BOLGE_KODU, BOLGE_ADI, SUBE_KODU, SUBE_ADI, count(distinct PORTFOLIO_ID) as PORTFOY_ADET, count(distinct MIY_SICIL) as MIY_ADET, count(*) as MUSTERI_ADET
    from KLV_MUSTERI_GNL2
    where BOLGE_KODU is not null
        and BOLGE_KODU!=1974
    group by BOLGE_KODU, BOLGE_ADI, SUBE_KODU, SUBE_ADI
    having count(distinct MIY_SICIL)>0
"""
sql = """
    select 
        b.COMMERCIAL_MKT_REG_ORG_NUM as BOLGE_KODU, b.COMMERCIAL_MKT_REG_ORG_NAME as BOLGE_ADI, 
        b.BRANCH_NUM as SUBE_KODU, b.BRANCH_NAME as SUBE_ADI, 
        count(distinct c.ORGANIZATION_ID) as PORTFOY_ADET,
        count(distinct d.USER_CODE) as MIY_ADET,
        count(*) as MUSTERI_ADET
    from pgaredm.v_cnf_customer_hst a
    left join PGAREDM.V_CNF_BRANCH_HST b 
        on  a.MAIN_BRANCH_NUM=b.BRANCH_NUM 
        and b.RECORD_DATE=to_date('30/09/2022','DD/MM/YYYY')
    left join PGAREDW.V_PRT_CUSTOMER_ORG_REL c 
        on  a.IP_ID=c.IP_ID 
        and c.END_DATE=to_date('01/01/9999','DD/MM/YYYY') 
        and ACTIVE_FLG=1 
        and RELATION_TP_ID=5
    left join PGAREDM.V_CNF_PORTFOLIO_HST d 
        on  substr(c.ORGANIZATION_ID,6)=d.PORTFOLIO_CODE
        and d.RECORD_DATE=to_date('30/09/2022','DD/MM/YYYY')
        and d.PORTFOLIO_ACTIVE_FLG=1
    where 
        a.record_date=to_date('30/09/2022','DD/MM/YYYY')
        and a.CUSTOMER_STATUS_CODE='A'
        and b.BRANCH_NUM is not null
        and b.COMMERCIAL_MKT_REG_ORG_NUM is not null
    group by
        b.COMMERCIAL_MKT_REG_ORG_NUM, b.COMMERCIAL_MKT_REG_ORG_NAME, 
        b.BRANCH_NUM, b.BRANCH_NAME

"""
conn = connect_db()
df = pd.read_sql(sql, conn)
df.head()

In [ ]:
display(df.sort_values('MUSTERI_ADET'))

In [ ]:
df.MUSTERI_ADET.hist(bins=50)
plt.show()

In [ ]:
df.MUSTERI_ADET[:30].plot(kind='bar', figsize=(10,4))
plt.show()

#### bar chart

In [ ]:
dfg = df.groupby('BOLGE_ADI')['MUSTERI_ADET'].sum().sort_values()
dfg

In [ ]:
dfg = df.groupby('BOLGE_ADI')['MUSTERI_ADET'].sum().sort_values()

# chart oluştur
fig, ax = plt.subplots(figsize=(16,4))

# yeni bir grafik
ax.bar(x=dfg.index, height=dfg.values)

# grafik özelliklerini düzenle
ax.set_xticks( np.arange(len(dfg.index)), dfg.index, rotation=45, ha='right',)

# grafiği göster
plt.show()


In [ ]:
from matplotlib.text import Text

In [ ]:
fig, ax = plt.subplots(figsize=(16,7))

ax.bar(x=dfg.index, height=dfg, label='Müşteri Adet x')

ax.set_xticks( np.arange(len(dfg.index)), dfg.index, rotation=45,   ha='right')
#ax.spines['top'].set_visible(False)
#ax.spines['left'].set_visible(False)
#ax.spines['right'].set_visible(False)
[ax.spines[s].set_visible(False) for s in ('left','top','right')]
ax.grid(True, axis='y')
ax.set_title('Müşteri Adetleri')
ax.set_xlabel('Bölge Adı')
ax.set_ylabel('Müşteri Adet')
ax.legend()

plt.tight_layout()
plt.savefig('plot1.png')
plt.show()


#### çoklu veri gösterimi

In [ ]:
# chart oluşturma
fig, ax = plt.subplots(figsize=(16,5))
fig.tight_layout()

# veri gösterimi
ax.bar(x=dfg.index, height=dfg, label='Müşteri Adet')
ax.axhline(y=dfg.mean(), label='Ort. Adet', c='red', linestyle='--')

# görsel düzenlemeler
ax.set_xticks( np.arange(len(dfg.index)), dfg.index, rotation=45, ha='right')
[ax.spines[s].set_visible(False) for s in ('left','top','right')]
ax.grid(True, axis='y')
ax.set_title('Müşteri Adetleri')
ax.set_xlabel('Bölge Adı')
ax.set_ylabel('Müşteri Adet')
ax.legend()

# gösterim
plt.show()

#### çoklu chart gösterimi

In [ ]:
# chart oluşturma
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16,5) )
fig.tight_layout()

# veri gösterimi, chart1
ax[0].bar(x=dfg.index, height=dfg, label='Müşteri Adet')
ax[0].axhline(y=dfg.mean(),  label='Bölge Ort.', c='red')

# görsel düzenlemeler, chart1
ax[0].set_xticks( np.arange(len(dfg.index)), dfg.index, rotation=45, ha='right')
[ax[0].spines[s].set_visible(False) for s in ('left','top','right')]
ax[0].grid(True, axis='y')
ax[0].set_title('Bölge Müşteri Adetleri')
ax[0].set_xlabel('Bölge Adı')
ax[0].set_ylabel('Müşteri Adet')
ax[0].legend()

# veri gösterimi, chart2
ax[1].hist(df.MUSTERI_ADET, edgecolor='white', alpha=0.7)
ax[1].axvline(x=df.MUSTERI_ADET.median(),  c='red', label='Şubeler Ort', linestyle='--')

# görsel düzenlemeler, chart1
[ax[1].spines[s].set_visible(False) for s in ('left','top','right')]
ax[1].grid(True, axis='y')
ax[1].set_title('Müşteri Adet Dağılımı')
ax[1].legend()

# gösterim
plt.suptitle('Müşteri Adet Analizi', y=1.10)
plt.show()

### subplot_mosaic

https://matplotlib.org/stable/tutorials/provisional/mosaic.html

In [ ]:
# chart oluşturma
"""
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(16,10), 
                       gridspec_kw={'height_ratios':[1,1],
                                    'width_ratios': [3,2]
                                   }, )
"""
ax = plt.figure(figsize=(16,10), tight_layout=True).subplot_mosaic(
    [
        [0,0,1],
        [2,2,2]
    ],
    gridspec_kw={'height_ratios':[3,2]}
)
fig.tight_layout()

# veri gösterimi, chart1
ax[0].bar(x=dfg.index, height=dfg, label='Müşteri Adet')
ax[0].axhline(y=dfg.mean(), label='Bölge Ort.', c='red')

# görsel düzenlemeler, chart sol-üst
ax[0].set_xticks( np.arange(len(dfg.index)), dfg.index, rotation=45, ha='right')
[ax[0].spines[s].set_visible(False) for s in ('left','top','right')]
ax[0].grid(True, axis='y')
ax[0].set_title('Bölge Müşteri Adetleri')
ax[0].set_xlabel('Bölge Adı')
ax[0].set_ylabel('Müşteri Adet')
ax[0].legend()

# görsel düzenlemeler, chart sağ-üst
pie_df = dfg.sort_values(ascending=False)
pie_df = pd.concat([pie_df.head(5), pd.Series(pie_df.iloc[6:].sum(), index=['Diğerleri'])])
ax[1].pie(x=pie_df.values, 
            labels=pie_df.index,
            autopct='%1.2f%%', pctdistance=0.8, labeldistance=1.2
           )
ax[1].set_title('İlk 5 Bölge')


# veri gösterimi, chart sol-alt


# veri gösterimi, chart sağ-alt
ax[2].hist(df.MUSTERI_ADET, edgecolor='white', alpha=0.7)
ax[2].axvline(x=df.MUSTERI_ADET.median(),  c='red', label='Şubeler Ort', linestyle='--')


# görsel düzenlemeler, chart1
[ax[2].spines[s].set_visible(False) for s in ('left','top','right')]
ax[2].grid(True, axis='y')
ax[2].set_title('Müşteri Adet Dağılımı')
ax[2].legend()

plt.subplots_adjust(left=0.125,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.2, 
                    hspace=0.50)

# gösterim
plt.suptitle('Müşteri Adet Analizi')
plt.show()


In [ ]:
axt = ax[1]

In [ ]:
axt.pie()